In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

In [ ]:
# sw_lat = 50.0500
# sw_lng = 20.0000
# ne_lat = 50.1000
# ne_lng = 20.1000

sw_lat = 53.477119
sw_lng = 9.735120
ne_lat = 46.937727
ne_lng = 23.621838

In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={API_KEY}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance = element['distance']['text']
            duration = element['duration']['text']
            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

In [ ]:
center_latitude, center_longitude = find_center(sw_lat, sw_lng, ne_lat, ne_lng)
# center_latitude = 48.201134
# center_longitude = 16.377229
road_points = [snap_to_road(center_latitude, center_longitude, API_KEY)]
original_points = [(center_latitude, center_longitude)]
attempts = 0
number_of_clients = 15

while len(road_points) < number_of_clients+1:
    attempts += 1
    point = generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng)
    road_point = snap_to_road(point[0], point[1], API_KEY)

    if road_point:
        road_points.append(road_point)
        original_points.append(point)
        print(f"Point {len(road_points)}: Original = {point[0]}, {point[1]}; Snapped = {road_point[0]}, {road_point[1]}")
    else:
          print(f"Attempt {attempts}: No road found nearby for Latitude = {point[0]}, Longitude = {point[1]}")

data = []

for i in range(len(road_points)):
    row = {
        'Original Latitude': original_points[i][0],
        'Original Longitude': original_points[i][1],
        'Snapped Latitude': road_points[i][0],
        'Snapped Longitude': road_points[i][1]
    }

    for j in range(len(road_points)):
        if i == j:
            row[f'Real Distance to Point {j + 1}'] = '0'
            row[f'Time to Point {j + 1}'] = '0'
            row[f'Road Distance to Point {j + 1}'] = '0'
        elif j > i:
            straight_distance = geodesic(road_points[i], road_points[j]).kilometers
            road_distance, travel_time = get_road_distance_time(road_points[i], road_points[j], API_KEY)

            row[f'Real Distance to Point {j + 1}'] = f'{straight_distance:.2f}'
            row[f'Time to Point {j + 1}'] = travel_time if travel_time else 'N/A'
            row[f'Road Distance to Point {j + 1}'] = road_distance if road_distance else 'N/A'
        elif i > j:
            straight_distance = geodesic(road_points[j], road_points[i]).kilometers
            road_distance, travel_time = get_road_distance_time(road_points[j], road_points[i], API_KEY)

            row[f'Real Distance to Point {j + 1}'] = f'{straight_distance:.2f}'
            row[f'Time to Point {j + 1}'] = travel_time if travel_time else 'N/A'
            row[f'Road Distance to Point {j + 1}'] = road_distance if road_distance else 'N/A'

    print(f"Data snaped for point {i + 1}")

    data.append(row)

df = pd.DataFrame(data)